
# Case Study: AAVE — چارچوب تحلیل فاندامنتال

> این نوت‌بوک یک الگوی اجرایی است تا بتوانید پروژه‌های مختلف را با ورودی‌های شفاف تحلیل کنید. هیچ‌کدام از اعداد پیش‌فرض «سیگنال» یا «عدد قطعی» نیستند.


In [ ]:
# وارد کردن کتابخانه‌ها
import os, yaml, math
from dataclasses import dataclass, asdict
from typing import Dict, Any

DATASET_PATH = "../datasets/project_profiles.yaml"

with open(DATASET_PATH, "r", encoding="utf-8") as f:
    data = yaml.safe_load(f)

projects = {p["id"]: p for p in data["projects"]}
list(projects.keys()), projects.get("aave", {})["name"]



## 1) بارگذاری پروفایل پروژه


In [ ]:
project_id = "aave"  # می‌توانید این را به پروژه‌ی دیگری تغییر دهید (پس از افزودن به YAML)
project = projects[project_id]
project



## 2) چارچوب امتیازدهی — وزن‌ها
وزن‌های نمونه برای سه لایه‌ی اصلی.


In [ ]:
weights = {
    "tokenomics": 0.4,
    "team_investors": 0.3,
    "usecase_valuation": 0.3,
}
sum(weights.values())



## 3) امتیازدهی مؤلفه‌ها
امتیازها را با تکیه بر شواهد/داده و چک‌لیست‌های ماژول‌ها تعیین کنید.


In [ ]:
scores = {
    "tokenomics": 75,       # TODO: با تحلیل واقعی جایگزین شود
    "team_investors": 80,   # TODO
    "usecase_valuation": 70 # TODO
}

final_score = sum(scores[k] * weights[k] for k in weights)
final_score



## 4) مدل‌سازی ارزش — قالب ساده‌ی سناریو
این بخش صرفاً الگوست. ورودی‌ها را با داده‌های خودتان جایگزین کنید.


In [ ]:
@dataclass
class Scenario:
    name: str
    annual_protocol_fees: float     # درآمد سالانه پروتکل (به دلار)
    tokenholder_share: float        # سهمی که به توکن می‌رسد (۰..۱)
    discount_rate: float            # نرخ تنزیل (۰..۱)
    growth_years: int               # افق رشد (سال)
    terminal_growth: float          # رشد بلندمدت (۰..۱)
    effective_supply: float         # عرضه‌ی موثر توکن (تعداد)

def dcf_value(s: Scenario):
    # مدل DCF ساده با رشد تا افق و سپس رشد نهایی
    cf0 = s.annual_protocol_fees * s.tokenholder_share
    pv = 0.0
    for t in range(1, s.growth_years + 1):
        cft = cf0 * ((1 + s.terminal_growth) ** (t-1))
        pv += cft / ((1 + s.discount_rate) ** t)
    # ارزش ادامه (Gordon Growth) در انتهای سال growth_years
    cfn = cf0 * ((1 + s.terminal_growth) ** s.growth_years)
    terminal_value = cfn * (1 + s.terminal_growth) / (s.discount_rate - s.terminal_growth)
    pv += terminal_value / ((1 + s.discount_rate) ** s.growth_years)
    token_fair_price = pv / s.effective_supply
    return {"PV": pv, "token_fair_price": token_fair_price}

scenarios = [
    Scenario("بدبینانه", annual_protocol_fees=50_000_000, tokenholder_share=0.1, discount_rate=0.25, growth_years=5, terminal_growth=0.02, effective_supply=16_000_000),
    Scenario("پایه",     annual_protocol_fees=120_000_000, tokenholder_share=0.15, discount_rate=0.22, growth_years=5, terminal_growth=0.02, effective_supply=16_000_000),
    Scenario("خوش‌بینانه", annual_protocol_fees=250_000_000, tokenholder_share=0.2, discount_rate=0.20, growth_years=5, terminal_growth=0.02, effective_supply=16_000_000),
]

results = {s.name: dcf_value(s) for s in scenarios}
results



## 5) حساسیت‌سنجی کوتاه


In [ ]:
def sensitivity(annual_protocol_fees_list, tokenholder_share_list, effective_supply=16_000_000):
    table = []
    for f in annual_protocol_fees_list:
        for sh in tokenholder_share_list:
            s = Scenario("sens", f, sh, 0.22, 5, 0.02, effective_supply)
            out = dcf_value(s)
            table.append({
                "fees": f,
                "share": sh,
                "fair_price": out["token_fair_price"]
            })
    return table

sens_table = sensitivity(
    annual_protocol_fees_list=[60_000_000, 120_000_000, 200_000_000],
    tokenholder_share_list=[0.1, 0.15, 0.2]
)
sens_table[:5]



## 6) خروجی نهایی


In [ ]:
output = {
    "project": project["name"],
    "scores": scores,
    "weights": weights,
    "final_score": final_score,
    "dcf_results": results,
    "timestamp": str(__import__("datetime").datetime.utcnow())
}
output
